# Agenda

1.  Plotting Ecosystem in Python

2.  Line plots in Matplotlib and Seaborn

3.  Seaborn Overview

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

-----

## Plotting Ecosystem in Python

We'll be using the visualization functionality of three packages:

- `matplotlib`: The workhorse engine of static plots in Python, based on Matlab's approach to visualization.

- `pandas`: Though not it's focus, the package includes wrappers for `matplotlib`: just call `.plot()` method on a data frame and ... see what you get. Good for quick and dirty visualization.

- `seaborn`: A higher-level wrapper for `matplotlib`.

    - More polished default aesthetics.
    - Uses `ggplot2` style non-standard evaluation.
    - Prefers tall tidy data frames (also like `ggplot2`)

When working with these packages, it's good to keep in mind the two different interfaces that they provide.

### Active Interface

Consider the following code.

In [ ]:
plt.figure()  # create a plot figure

# create the first of two panels
plt.subplot(2, 1, 1) # (rows, columns, panel number)
plt.plot(np.random.normal(scale = 1, size = 100));

# create the second panel
plt.subplot(2, 1, 2)
plt.plot(np.random.normal(scale = 2, size = 100));

In this interface, `matplotlib` is keeping track of which plot (or subplot) is the active one. Any plotting commands called will update that plot. This is similar to base Rs approach. This interface is brittle and error-prone. Better to use...

### Object-oriented Interface

In [ ]:
# First create a grid of plots
# ax will be an array of two Axes objects
fig, ax = plt.subplots(2)

# Call plot() method on the appropriate object
ax[0].plot(np.random.normal(scale = 1, size = 100));
ax[1].plot(np.random.normal(scale = 2, size = 100));

Here, we're calling the `.plot()` method on a particulate `ax` object (a subplot), so we can be explicit in what we're modifying. `plt.plot()` , by contrast, will modify whatever the active plot is.

## Simple line plot

### First in `matplotlib`

When building a plot, most functions will work either on the *figure* or the *axes*.

In the plot below, we first create each object separately. To plot a function along the axis, we call the `.plot()` method on that ax (essentially the panel inside the "canvas" of the figure).

In [ ]:
fig = plt.figure()
ax = plt.axes()
x = np.linspace(0, 10, 1000)
ax.plot(x, np.sin(x));
# plt.plot(x, np.sin(x)) # "active" interface approach

To plot multiple lines on the same axes, you can call the `.plot()` method again with a different function.

In [ ]:
fig = plt.figure()
ax = plt.axes()
x = np.linspace(0, 10, 1000)
ax.plot(x, np.sin(x));
ax.plot(x, np.cos(x));

#### Fine tuning the aesthetics

Several aesthetic preferences can be passed as arguments to `.plot()` including `color` and `linetype`. Test out each of the different approaches to calling `.plot()` to see how the color and linetype responds.

In [ ]:
fig = plt.figure()
ax = plt.axes()
x = np.linspace(0, 10, 1000)
ax.plot(x, np.sin(x), color = "g", linestyle = "dashed");
#ax.plot(x, np.sin(x), color = "blue", linestyle = "-.");
#ax.plot(x, np.sin(x), "-.g");

Other aesthetics are specified through additional method or function calls, one for each edit. This can be done either using the active plot or OOP approach. A few common functions are found below. The OOP approach often, but not always, prefaces the method with `set_`.

**OOP**

-   `ax.set_xlabel()`
-   `ax.set_ylabel()`
-   `ax.set_xlim()`
-   `ax.set_ylim()`
-   `ax.set_title()`

**Active Plot**

-   `plt.xlabel()`
-   `plt.ylabel()`
-   `plt.xlim()`
-   `plt.ylim()`
-   `plt.title()`

### Example 1: Cumulative ballot returns

<img src="https://github.com/andrewpbray/python-for-r-users/blob/master/labs/figs/or-plot-1.png?raw=true" width="400"/>

Let's recreate this plot from the Reshaping Votes lab. First we prepare the data,

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/andrewpbray/python-for-r-users/master/data/or_df_1.csv")
df = df.loc[:35, :]
df = df.rename(columns = {"Statewide Ballot Returns by": "County"})
df2 = df.melt(id_vars = "County",
        value_vars = ['Oct 16', 'Oct 19', 'Oct 20', 'Oct 21', 'Oct 22', 'Oct 23',
       'Oct 26', 'Oct 27', 'Oct 28', 'Oct 29', 'Oct 30', 'Nov 2', 'Nov 3'],
        var_name = "Date",
        value_name = "Count")
df2 = df2.replace(',','', regex = True)
df2["Count"] = pd.to_numeric(df2["Count"])
df2["Date"]  = pd.to_datetime(df2["Date"] + " 2020", format = "%b %d %Y")
df2["Cumulative count"] = df2[["Count", "County"]].groupby("County").cumsum()
mult = df2[df2["County"] == "Multnomah"]
des = df2[df2["County"] == "Deschutes"]

and then construct the plot.

#### OOP approach

In [ ]:
fig, axes = plt.subplots(1, 2)
axes[0].tick_params("x", labelrotation = 90)
axes[0].plot("Date", "Cumulative count", data = mult); # alternate format (ggploty!)
axes[1].tick_params("x", labelrotation = 90)
axes[1].plot(des["Date"], des["Cumulative count"]);

Note that the first plot was constructed by passing the strings of the variable names and then the data frame separately. This is called *non-standard evaluation* and is what is used extensively in the R tidyverse and in `seaborn`.

We still have some ugliness there:

-   Date are cut off at bottom
-   No labels on axes or figure

Let's see if we can fix it.

In [ ]:
fig, axes = plt.subplots(1, 2)
axes[0].tick_params("x", labelrotation = 90)
axes[0].plot("Date", "Cumulative count", data = mult); # alternate format (ggploty!)
axes[1].tick_params("x", labelrotation = 90)
axes[1].plot(des["Date"], des["Cumulative count"]);
plt.subplots_adjust(bottom = .2, wspace = .3);
axes[0].set_xlabel("Date")
axes[1].set_xlabel("Date")
axes[0].set_title("Multnomah County")
axes[1].set_title("Deschutes County")
fig.suptitle("Cumulative Ballot Returns");

It's still not perfect - it'd be nice to get those date labels looking better - but it'll do for now. If you're interested in how to tinker with the way date labels are printed, see the docs on [date tick labels](https://matplotlib.org/3.1.1/gallery/text_labels_and_annotations/date.html).

#### Active figure approach

In [ ]:
plt.figure()
plt.subplot(1, 2, 1)
plt.plot("Date", "Cumulative count", data = mult);
plt.tick_params("x", labelrotation = 90)
plt.xlabel("Date")
plt.title("Multnomah County")
plt.subplot(1, 2, 2)
plt.plot("Date", "Cumulative count", data = des);
plt.tick_params("x", labelrotation = 90)
plt.xlabel("Date")
plt.title("Deschutes County")
plt.subplots_adjust(bottom = .2, wspace = .3)
fig.suptitle("Cumulative Ballot Returns");

### Second, in `seaborn`

Seaborn adds two useful things to the mix:

1.  More fully featured plot functions.
2.  More polished aesthetics

We can use `sns.lineplot()` to recreate the line plot for just Multnomah County (active figure interface),

In [ ]:
sns.lineplot(data = mult, x = "Date", y = "Cumulative count")
plt.tick_params("x", labelrotation = 90)
plt.title("Multnomah County")
plt.subplots_adjust(bottom = .3)

and we can also use the OOP interface:

In [ ]:
fig = plt.figure()
ax = sns.lineplot(data = mult, x = "Date", y = "Cumulative count")
ax.tick_params("x", labelrotation = 90)
ax.set_title("Multnomah County")
fig.subplots_adjust(bottom = .3)

Note that both forms use non-standard evaluation, where you pass the name of the data frame, then refer to the variables simply as strings.

We can plot both counties much more elegantly and efficiently using faceting.

In [ ]:
mult_des = df2[df2["County"].isin(["Multnomah", "Deschutes"])]
g = sns.FacetGrid(data = mult_des, col = "County", margin_titles = True)
g.map(sns.lineplot, "Date", "Cumulative count");

This is a *big* improvement over the way that we made this plot in `matplotlib`. That code involved splitting out data set out into two separate data sets. This allows us to keep them well-organized in a single data set.

We can get the same thing by using a higher-level wrapper function called `.relplot()` (relational plot). We'll also use functions very similar to the ones we used in `matplotlib` to fine tune the aesthetics (label rotation, spacing, and a title).

In [ ]:
g = sns.relplot(data = mult_des, 
              x = "Date", 
              y = "Cumulative count",
              col = "County",
              kind = "line")
g.set_xticklabels(rotation = 90)
g.fig.subplots_adjust(bottom = .3)
g.fig.suptitle("Cumulative Ballot Returns");

`.relplot()` allows us to make `kind = "line"` and `kind = "scatter"` plots with more complex functionality (like faceting) in a single function call.

#### Question 1

Use `sns.relplot()` to create a scatter plot of bill length on bill depth with faceting based on species.

In [ ]:
# answer

### Example 2: Cumulative ballot returns for all counties

<img src="https://github.com/andrewpbray/python-for-r-users/blob/master/labs/figs/or-plot-2.png?raw=true" width="300"/>

This is a (very busy) way to communicate the ballot return data from all counties in a single plot. Let's first build it using `matbplotlib`. We start by getting each of the county cumulative counts along their own column.

In [ ]:
df3 = df2.pivot(index = "Date",
                columns = "County", 
                values = "Cumulative count")
df3

To get all of those lines on to the same plot requires calling `.plot()` many times. This is best done in a for loop.

In [ ]:
fig, axes = plt.subplots(1, 1)

num = 0
for column in df3:
    num += 1
    plt.plot(df3.index, 
        df3[column], 
        marker = '', 
        color = "gray",
        linewidth = 1,
        alpha = 0.9, 
        label = column)


#### Question 2

Use `sns.relplot()` or `sns.lineplot()` to recreate this plot.

In [ ]:
# answer

## Seaborn Overview

<img src="https://github.com/andrewpbray/python-for-r-users/blob/master/lectures/figs/seaborn-overview.png?raw=true" width="600"/>

As we said at the outset, the main advantage you get with `seaborn` is access to several high-level functions that save you from having to build-up plots "by hand" in `matplotlib`. In fact, each of these powerful plotting functions can alternatively be called from their even higher-level wrapper: `relplot()`, `displot()`, and `catplot()`.